# SEA Petrochemical Industry

Here, we analyze petrochemical project financing in Southeast Asia, based on data pulled from LSEG Refinitiv.

In [1]:
# import libraries
import pandas as pd
import os

In [2]:
# read files

pf_projects_df = pd.read_excel("wf_data.xlsx", sheet_name="pf_projects")
pf_developers_df = pd.read_excel("wf_data.xlsx", sheet_name="pf_developers")

In [3]:
# setting up folder for data
folder_name = "pf_data"
os.makedirs(folder_name, exist_ok=True)

## Project financing, pt. 1: Country of the project

In [4]:
pf_projects_df.dtypes

pf_project_id                              object
sdc_deal_number_1                           int64
sdc_deal_number_2                           int64
announced_date                     datetime64[ns]
project_name                               object
project_country                            object
sector                                     object
subsector                                  object
output                                     object
type                                       object
project_status                             object
financing_status                           object
financed_date                      datetime64[ns]
financing_subcategory                      object
borrower                                   object
total_cost_usd                            float64
is_total_cost_estimated                      bool
govt_support                               object
support_type                               object
synopsis                                   object


In [5]:
# automates breakdown of projects

def proj_summary(
    df,
    project_name_col="project_name", 
    project_id_col="project_id", 
    country_col="project_country", 
    total_cost_col="project_cost", 
    status_col="project_status"
    ):
    """
    This function counts the total number of projects and cancelled projects.
    It counts the projects by country and sums estimated total costs.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing project information
    project_name_col (str): column name for project name
    project_id_col (str): column name for project ID
    country_col (str): column name for project country
    total_cost_col (str): column name for total project cost
    status_col (str): column name for project status

    Returns:
    df[0] = cancelled_projects: DataFrame of cancelled projects
    df[1] = per_country_analysis: DataFrame of number of projects, total cost, and average project cost by country
    """

    # counts unique project identifiers
    # to get the total number of projects
    total_projects = df[project_id_col].nunique()
    print(f"The total number of projects are: {total_projects}")

    # counts cancelled projects
    cancelled_projects = df[df[status_col].str.lower().isin(["cancel", "canceled", "cancelled"])]
    print(f"Cancellations: {len(cancelled_projects)} of the {total_projects} total projects were cancelled")
    
    # counts projects and sums costs by country
    per_country_analysis = df.groupby(country_col).agg(
        number_of_projects = (project_id_col, "count"),  # count projects
        total_cost = (total_cost_col, "sum")  # sum of total cost
    ).assign(
        average_cost_per_project = lambda x: x["total_cost"] / x["number_of_projects"]
    ).sort_values(by="number_of_projects", ascending=False)

    print("Per country breakdown:")
    print(per_country_analysis)
    return cancelled_projects, per_country_analysis

In [6]:
pf_result_df = proj_summary(pf_projects_df,
                      project_name_col='project_name', 
                      project_id_col='sdc_deal_number_1', 
                      country_col='project_country', 
                      total_cost_col='total_cost_usd', 
                      status_col='project_status')

The total number of projects are: 69
Cancellations: 2 of the 69 total projects were cancelled
Per country breakdown:
                 number_of_projects    total_cost  average_cost_per_project
project_country                                                            
Indonesia                        28  38937.350000               1390.619643
Malaysia                         13   3370.051886                259.234760
Singapore                        10    227.810000                 22.781000
Thailand                          6  17918.030000               2986.338333
Vietnam                           4  23330.880000               5832.720000
Philippines                       3   1190.480000                396.826667
Cambodia                          2     25.870000                 12.935000
Myanmar                           2   6000.000000               3000.000000
Laos                              1      0.000000                  0.000000


In [7]:
pf_result_df[1].to_csv("pf_data/pf_count.csv", encoding="UTF-8")

### Counting projects by identified subsector

In [8]:
# make everything lowercase for ease of string search
pf_projects_df['operations_type'] = pf_projects_df['operations_type'].str.lower()

# count occurences
pf_ops_counts = pf_projects_df['operations_type'].value_counts()

# count per type
pf_upstream = pf_ops_counts.get("upstream")
pf_midstream = pf_ops_counts.get("midstream")
pf_downstream = pf_ops_counts.get("downstream")
pf_mixed = pf_projects_df[pf_projects_df["operations_type"].str.contains("/")].shape[0]

print("Upstream projects:", pf_upstream)
print("Midstream projects:", pf_midstream)
print("Downstream projects:", pf_downstream)
print("Mixed projects:", pf_mixed)

Upstream projects: 2
Midstream projects: 22
Downstream projects: 40
Mixed projects: 5


## Project financing, pt. 2: Country of the developer

In [9]:
pf_developers_df.dtypes

pf_developer_id               object
pf_project_001                object
sdc_deal_number_1              int64
sdc_deal_number_2              int64
project_name                  object
project_country               object
developer                     object
developer_country             object
sponsor_pct                  float64
developer_parent              object
developer_parent_country      object
project_company               object
project_company_ownership     object
dtype: object

In [10]:
# automates breakdown of countries / state-owned

def proj_dev_summary(
    df,
    project_name_col="project_name", 
    project_id_col="project_id", 
    country_col="developer_parent_country",
    developer_parent_col="developer_parent",
    developer_name_col="developer_name"
    ):
    """
    This function filters only state-owned developers and counts the projects by country.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing project information
    project_name_col (str): column name for project name
    project_id_col (str): column name for project ID
    country_col (str): column name for project country
    developer_parent_col (str): column name for the parent of the developer, will show whether state-owned or otherwise
    developer_name_col (str): column name for developer name

    Returns:
    df[0] = state_owned_df: DataFrame of state-owned project developers
    df[1] = state_owned_analysis: DataFrame of number of projects per parent company of developer and by country
    """

    # filters state-owned developers
    state_owned_df = df[df[developer_parent_col].str.contains("state", case=False, na=False)]

    # groups by country and developer
    state_owned_analysis = state_owned_df.groupby([country_col, developer_parent_col]).size().reset_index(name='project_count')
    return state_owned_df, state_owned_analysis


In [11]:
pf_state_owned = proj_dev_summary(
    pf_developers_df,
    project_name_col="project_name", 
    project_id_col="sdc_deal_number_1", 
    country_col="developer_parent_country",
    developer_parent_col="developer_parent",
    developer_name_col="developer"
)

pf_state_df = pf_state_owned[1]
pf_state_df["developer_parent"] = pf_state_df["developer_parent"].str.replace("State-owned [", "").str.replace("]", "")
pf_state_df

,developer_parent_country,developer_parent,project_count
0,China,PetroChina,1
1,Indonesia,PLN,4
2,Indonesia,Pertamina,11
3,Indonesia,Pupuk,6
4,Malaysia,Petronas,2
5,Malaysia,SEDC,4
6,Philippines,DOE,1
7,Saudi Arabia,Saudi Arabian Oil Co,1
8,Singapore,EDB,1
9,Singapore,EMA,1


In [12]:
pf_state_df.to_csv("pf_data/pf_devs.csv", encoding="UTF-8", index=False)

In [13]:
# counting per country
pf_state_df.groupby("developer_parent_country")["project_count"].sum().sort_values(ascending=False)

developer_parent_country
Indonesia               21
Malaysia                 6
Singapore                6
Thailand                 4
South Korea              2
Vietnam                  2
China                    1
Philippines              1
Saudi Arabia             1
Taiwan                   1
United Arab Emirates     1
Name: project_count, dtype: int64

In [14]:
# counting countries
pf_state_df["developer_parent_country"].value_counts()

developer_parent_country
Singapore               4
Indonesia               3
Thailand                3
Malaysia                2
China                   1
Philippines             1
Saudi Arabia            1
South Korea             1
Taiwan                  1
United Arab Emirates    1
Vietnam                 1
Name: count, dtype: int64

### Project financing, pt. 3: False solutions

In [15]:
pf_projects_df.dtypes

pf_project_id                              object
sdc_deal_number_1                           int64
sdc_deal_number_2                           int64
announced_date                     datetime64[ns]
project_name                               object
project_country                            object
sector                                     object
subsector                                  object
output                                     object
type                                       object
project_status                             object
financing_status                           object
financed_date                      datetime64[ns]
financing_subcategory                      object
borrower                                   object
total_cost_usd                            float64
is_total_cost_estimated                      bool
govt_support                               object
support_type                               object
synopsis                                   object


In [16]:
# automates analysis

def keyword_summary(
    keyword_search,
    df,
    project_name_col="project_name", 
    project_id_col="project_id", 
    project_desc_col="description",
    country_col="country"
    ):
    """
    This function searches the project description column for a keyword.
    Use case: finding projects that involves hydrogen and ammonia production, and/or carbon capture utilization and storage
    
    Parameters:
    keyword_search (str or list): word or phrase to search in the project description
    df (pandas.DataFrame): DataFrame containing project information
    project_name_col (str): column name for project name
    project_id_col (str): column name for project ID
    project_desc_col (str): column name for project description
    country_col (str): column name for project country

    Returns:
    df[0] = search_df: DataFrame of projects with keywords filtered
    df[1] = per_country_analysis: DataFrame of number of total projects, projects with keywords filtered by country
    """

    # searches string keyword
    keyword_pattern = '|'.join(keyword_search) # for multiple keywords
    search_df = df[df[project_desc_col].str.contains(keyword_pattern, case=False, na=False)]

    # count keyword matches and total projects per country
    per_country_analysis = pd.DataFrame({
        "keyword_projects": search_df.groupby(country_col)[project_id_col].count(),
        "total_projects": df.groupby(country_col)[project_id_col].count()
    }).reset_index()

    # add percentage column
    per_country_analysis["percentage"] = (per_country_analysis["keyword_projects"] / 
                                    per_country_analysis["total_projects"] * 100).round(2)

    # sort by number of keyword projects in descending order
    per_country_analysis = per_country_analysis.sort_values("keyword_projects", ascending=False)

    print(f"There are {len(search_df)} projects found with the keywords {keyword_search}.")
    print("Per country breakdown:")
    print(per_country_analysis)
    return search_df, per_country_analysis


In [17]:
# search ammonia, hydrogen, or carbon capture

false_sol_df = keyword_summary(["ammonia", "hydrogen", "carbon capture"], pf_projects_df,
                       project_name_col="project_name",
                       project_id_col="sdc_deal_number_1",
                       project_desc_col="synopsis",
                       country_col="project_country"
                       )

false_sol_df[1]

There are 40 projects found with the keywords ['ammonia', 'hydrogen', 'carbon capture'].
Per country breakdown:
  project_country  keyword_projects  total_projects  percentage
1       Indonesia              16.0              28       57.14
3        Malaysia               9.0              13       69.23
6       Singapore               6.0              10       60.00
5     Philippines               3.0               3      100.00
7        Thailand               3.0               6       50.00
0        Cambodia               1.0               2       50.00
2            Laos               1.0               1      100.00
8         Vietnam               1.0               4       25.00
4         Myanmar               NaN               2         NaN


,project_country,keyword_projects,total_projects,percentage
1,Indonesia,16.0,28,57.14
3,Malaysia,9.0,13,69.23
6,Singapore,6.0,10,60.00
5,Philippines,3.0,3,100.00
7,Thailand,3.0,6,50.00
0,Cambodia,1.0,2,50.00
2,Laos,1.0,1,100.00
8,Vietnam,1.0,4,25.00
4,Myanmar,NaN,2,NaN


In [18]:
false_sol_df[1].to_csv("pf_data/pf_false_sol.csv", encoding="UTF-8", index=False)

In [19]:
# search hydrotreated vegetable oil or hvo

hvo_df = pf_projects_df[pf_projects_df["synopsis"].str.contains('hydrotreated|hvo', case=False, na=False)]
hvo_df["project_name"]

22     Pengerang Integrated Complex Biorefinery Project
23    ChemOne Holdings Pengerang Hydrotreated Vegeta...
41    550000 t/y Pulau Bukom Sustainable Aviation Fu...
Name: project_name, dtype: object

### The kinda tricky part...

Here, we will count the projects by developer, by country based on our filtered df.
What makes this tricky is that information on projects and developers are kept in separate dfs.

In [20]:
# first, we merge dfs
false_sol_dev_merged = false_sol_df[0].merge( ### false_sol_df[0] is the full list of filtered projects
    pf_developers_df, on="sdc_deal_number_1", how='left'
)

# then, simplify and drop columns not needed
columns_to_keep = [
    "sdc_deal_number_1", "project_name_x", "project_country_x", "developer_parent", "developer_parent_country"
]
false_sol_dev_selected = false_sol_dev_merged[columns_to_keep]
false_sol_dev_selected


,sdc_deal_number_1,project_name_x,project_country_x,developer_parent,developer_parent_country
0,154088689857,Hydrexia/WEDSB Selangor Hydrogen Oriented Reso...,Malaysia,Hydrexia Holding Ltd,China
1,154088689857,Hydrexia/WEDSB Selangor Hydrogen Oriented Reso...,Malaysia,Worldwide Holdings Berhad,Malaysia
2,154088589624,Penyu Basin Carbon Capture Storage Project,Malaysia,Petroliam Nasional Bhd,Malaysia
3,154088589624,Penyu Basin Carbon Capture Storage Project,Malaysia,State-owned [Abu Dhabi National Oil Co],United Arab Emirates
4,154088589624,Penyu Basin Carbon Capture Storage Project,Malaysia,Storegga Ltd,United Kingdom
...,...,...,...,...,...
89,154088367576,Sumitomo/ENEOS/SEDC Energy Sarawak Green Hydro...,Malaysia,Sumitomo Corp,Japan
90,154088367576,Sumitomo/ENEOS/SEDC Energy Sarawak Green Hydro...,Malaysia,ENEOS Holdings,Japan
91,154088367576,Sumitomo/ENEOS/SEDC Energy Sarawak Green Hydro...,Malaysia,State-owned [SEDC],Malaysia
92,154085559639,1200000 t/y Laos Potash Project,Laos,FYI Resources Ltd | Yuntianhua Group,Australia | China


In [21]:
false_sol_dev_selected.dtypes

sdc_deal_number_1            int64
project_name_x              object
project_country_x           object
developer_parent            object
developer_parent_country    object
dtype: object

In [22]:
# counting unique projects per developer
false_sol_dev_selected.groupby("developer_parent")["sdc_deal_number_1"].nunique().sort_values(ascending=False)

developer_parent
State-owned [Pertamina]      5
State-owned [PLN]            4
State-owned [Pupuk]          4
State-owned [Temasek]        3
Acwa Power Co                3
                            ..
MMC Group                    1
Aalborg CSP AS               1
PESTECH International Bhd    1
POSCO Co Ltd                 1
Worldwide Holdings Berhad    1
Name: sdc_deal_number_1, Length: 64, dtype: int64

In [23]:
# counting unique projects per country
false_sol_dev_selected.groupby("developer_parent_country")["sdc_deal_number_1"].nunique().sort_values(ascending=False)

developer_parent_country
Indonesia               14
Malaysia                10
Japan                    9
Singapore                7
United States            6
France                   6
Philippines              3
United Kingdom           3
Thailand                 3
Saudi Arabia             3
China                    2
South Korea              2
Sweden                   1
United Arab Emirates     1
Australia                1
Spain                    1
Australia | China        1
Netherlands              1
Italy                    1
Denmark                  1
Vietnam                  1
Name: sdc_deal_number_1, dtype: int64

In [24]:
false_sol_sorted = false_sol_dev_selected.groupby(["developer_parent_country", "developer_parent"])["sdc_deal_number_1"].nunique()\
.sort_values(ascending=False)

# false_sol_sorted.to_csv("pf-false-sol-devs.csv", encoding="UTF-8")

In [25]:
# false_sol_dev_selected.to_csv("pf-false-sol-selects.csv", encoding="UTF-8")